# **Top Trending Topics for Crude Oil**

This Jupyter notebook implements an **agentic workflow** based on the content retrieval from BigData API to **identify, verifiy, reindex, and summarize** the specialized news that are **trending topics** for the crude oil market.

The workflow is structured as follows:

**Step 1- Generation of the Lexicon**: Identify the specialized industry-specific jargon relevant to the crude oil market to ensure a high recall in the content retrieval.

**Step 2- Content Retrieval Based on BigData**: Perform a keyword search on the news content with the Bigdata API to retrieve documents, splitting the search over daily timeframes and multi-threading the content search on the individual keywords for speed purpose.

**Step 3- Topic Clustering and Selection**: Perform topic modelling using a large language model to verify and cluster the news. Then, the summarization ensures topic selection identifying the top trending news for crude oil, while deriving advanced analytics to quantify the trendiness (based on news volume), novelty (based on daily changes in summaries), impact and magnitude (based on the financial materiality on crude oil prices) of the trending topics.

**Step 4- Customized Report Generation**: Customize the ranking system of the summarized topics based on their trendiness, novelty, and financial materiality on crude oil prices, and display a daily market update. For verification purpose, the reports are supported by the granular news and sources.

**Output**

1. **Daily Market Reports**: A detailed and visually appealing report summarizing the top trending topics for crude oil, with a customizable ranking system to reindex the news.
2. **Actionable Dataframe**: A timestamped dataframe containing the granular news clustered into relevant topics, and the advanced analytics of trendiness, novelty, impact, and magnitude scores to be potentially used for backtesting purpose.

**Requirements**

- Credentials for the Bigdata API to perform keyword and document searches on news content.
- Credentials for the OpenAI API used in the notebook, this could be substituted with any other LLM.
- A `tools` folder in the same directory as this notebook, containing a Python file named `utils_reports.py` with all required functionalities.
- A `requirements.txt` file listing all the necessary Python libraries and dependencies. We recommend installing these packages in a virtual environment.

# Set-Up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(os.path.abspath("/home/abouchs/.python_env_var/.env"))

True

In [3]:
BIGDATA_USERNAME = os.getenv("BIGDATA_USERNAME")
BIGDATA_PASSWORD = os.getenv("BIGDATA_PASSWORD")

In [4]:
from bigdata_client import Bigdata
bigdata_cred = Bigdata(BIGDATA_USERNAME, BIGDATA_PASSWORD)

In [5]:
from src.lexicon_generator import LexiconGenerator
from src.search_topics import search_by_keywords
from src.topics_extractor import (process_all_reports,
                                run_process_all_trending_topics,
                                run_add_advanced_novelty_scores,
                                add_market_impact_to_df,
                                prepare_data_for_report,
                                generate_html_report)
from IPython.display import display
from IPython.core.display import HTML

In [6]:
output_dir = f"//home/abouchs/shared/OutputData/abouchs/Bigdata_cookbook/trending_topics/"

In [7]:
try:
    import asyncio
    asyncio.get_running_loop()
    import nest_asyncio; nest_asyncio.apply()
    print("✅ nest_asyncio applied")
except (RuntimeError, ImportError):
    print("✅ nest_asyncio not needed")

✅ nest_asyncio applied


# Step 1- Generation of the Lexicon

In this step, we identify the specialized industry-specific jargon relevant to the crude oil market to ensure a high recall in the content retrieval.

In [8]:
main_theme = "Crude Oil"
system_prompt = (
    f"""You are an expert tasked with generating a lexicon of the most important and relevant keywords specific to the given main theme and its related market.

    Your goal is to compile a list of terms that are critical for understanding and analyzing the main theme's market. This lexicon should include only the most essential keywords, phrases, and abbreviations that are directly associated with trading, analysis, logistics, and industry reporting related to the main theme.

    Guidelines:

    1. **Focus on relevance:** Include only the most important and commonly used keywords that are uniquely tied to main theme and its market. These should reflect key concepts, market mechanisms, pricing benchmarks, logistical aspects, and industry-specific terminology.
    2. **Avoid redundancy:** Do not repeat the word of the main theme or its components, such as "Crude" or "Oil" in multiple phrases. Include the main theme only as a standalone term, and focus on other specific terms without redundant repetition.
    3. **Strict exclusion of generic terms:** Exclude any terms that are generic or broadly used in other markets, such as "Arbitrage," "Hedge," "Liquidity," "Spot Price," "Futures Contract," "Backwardation," or "Contango," even if they have a specific meaning in the main theme market. Only include terms that are uniquely relevant to the main theme market and cannot be applied broadly.
    4. **Include specific variations:** Where applicable, provide both the full form and common abbreviations as SEPARATE keywords (e.g., "West Texas Intermediate" and "WTI" or variations like "Brent" and "Brent Crude").
    5. **Ensure clarity:** Each keyword should be concise, clear, and directly relevant to the main theme's market, avoiding any ambiguity.
    6. **Select only the most critical:** There is no need to reach a specific number of keywords. Focus solely on the most crucial terms without padding the list. If fewer keywords meet the criteria, that is acceptable.

    The output should be a lexicon of only the most critical and uniquely relevant keywords related to the main theme market, formatted as a JSON list.
    """
)


In [9]:
LexiconGenerator = LexiconGenerator(openai_key=os.getenv("OPENAI_API_KEY"), model="gpt-4o", seeds=[123, 123456, 123456789, 456789, 789])

In [10]:
keywords_lex = LexiconGenerator.generate(theme=main_theme, system_prompt=system_prompt)

# Step 2- Content Retrieval Based on Bigdata

In this section, we perform a keyword search on the news content with the Bigdata API to retrieve documents, splitting the search over daily timeframes and multi-threading the content search on the individual keywords for speed purpose. The user can define the time range below to generate daily reports between the start and end dates.

In [11]:
start_query = '2025-06-21'
end_query = '2025-06-28'

In [12]:
results, daily_keyword_count = search_by_keywords(
    keywords=keywords_lex,
    start_date=start_query,
    end_date=end_query,
    freq='D',
    document_limit=10)

About to run 840 queries
Example Query: Keyword('Brent') over date range: AbsoluteDateRange('2025-06-21T00:00:00', '2025-06-21T23:59:59')


Querying Bigdata...: 100%|██████████| 840/840 [02:54<00:00,  4.81it/s]


In [13]:
results

,timestamp,rp_document_id,headline,chunk_number,sentence_id,source_id,source_name,text,keyword,date
0,2025-06-21 00:00:00+00:00,0B4844B85EA092ED4AB50AB3534AB360,BW Energy: Update on Fixed Income Investor Mee...,3.0,0B4844B85EA092ED4AB50AB3534AB360-3,923B93,Financial Services Monitor Worldwide,BW Energy is a growth E&P company with a diffe...,E&P,2025-06-21
1,2025-06-21 00:00:00+00:00,10F391270D3AB34C66B069821928430F,"2 suspects arrested, drugs seized following No...",2.0,10F391270D3AB34C66B069821928430F-2,39E760,"Governance, Risk & Compliance Monitor Worldwide",Investigators reportedly found 26 grams of cra...,Cracking,2025-06-21
2,2025-06-21 00:00:00+00:00,5D552639AE35AB037AB8D54E1BB8E210,India's Core Sector Growth Drops to a Nine-Mon...,2.0,5D552639AE35AB037AB8D54E1BB8E210-2,923B93,Financial Services Monitor Worldwide,The ICI measures the combined and individual p...,Refinery,2025-06-21
3,2025-06-21 00:00:00+00:00,5DB7156ACE3425F4F61B3BE3E882AF0D,Premium Alcohol Market Forecasts Report 2025-2...,3.0,5DB7156ACE3425F4F61B3BE3E882AF0D-3,923B93,Financial Services Monitor Worldwide,Market Trends Growing Disposable Income and Ur...,Energy Information Administration,2025-06-21
4,2025-06-21 00:00:00+00:00,96EC510A80FAD37155CD1543681E8CC1,"Morning Briefing: June 21, 2025",11.0,96EC510A80FAD37155CD1543681E8CC1-11,CE1ADC,Anadolu Agency,China is the world's largest oil importer. In ...,Energy Information Administration,2025-06-21
...,...,...,...,...,...,...,...,...,...,...
11258,2025-06-28 23:39:31+00:00,0F75052C4F07615235AC9E79C4D05E44,Sharks SWING For The Upside With Their Day 2 P...,1.0,0F75052C4F07615235AC9E79C4D05E44-1,D1EEC6,KVUE News Online,The San Jose Sharks' 2025 NHL draft class has ...,Pipeline,2025-06-28
11259,2025-06-28 23:48:49+00:00,456DB9D74C8F0B01626EE845F8FF4CA6,Caught on camera: Car literally drives through...,2.0,456DB9D74C8F0B01626EE845F8FF4CA6-2,E54C73,ABC News,"""This is like a movie or something,"" Patel sai...",Barrel,2025-06-28
11260,2025-06-28 23:50:36+00:00,DF667772C27122819087C82C1D54C3DD,The Strategic Empire: Debt & the Dollar,47.0,DF667772C27122819087C82C1D54C3DD-47,EC0C87,Michael Hudson,"However, the condition for letting OPEC countr...",OPEC+,2025-06-28
11261,2025-06-28 23:50:36+00:00,DF667772C27122819087C82C1D54C3DD,The Strategic Empire: Debt & the Dollar,55.0,DF667772C27122819087C82C1D54C3DD-55,EC0C87,Michael Hudson,"Yes, someday the United States cannot get a fr...",OPEC+,2025-06-28


# Step 3- Topic Clustering and Selection

In this step, we perform topic modelling using a large language model to verify and cluster the news. Then, the summarization ensures topic selection identifying the top trending news for crude oil, while deriving advanced analytics to quantify the trendiness (based on news volume), novelty (based on daily changes in summaries), impact and magnitude (based on the financial materiality on crude oil prices) of the trending topics.

Before performing the topic clustering, we apply a verification layer to remove the news that are not relative to the oil market

In [14]:
model = "gpt-4o-mini" 
api_key = os.getenv("OPENAI_API_KEY")

In [15]:
semaphore_size = 1000

# Assuming unique_reports is your DataFrame
filtered_reports = process_all_reports(results, model, api_key, main_theme, semaphore_size)

Filtering News:   0%|          | 0/11263 [00:00<?, ?it/s]

In this cell, we leverage a LLM to perform topic modeling, identifying and clustering the key topics from the news reports.

In [16]:
filtered_reports

,timestamp,rp_document_id,headline,chunk_number,sentence_id,source_id,source_name,text,keyword,date
0,2025-06-21 00:00:00+00:00,10F391270D3AB34C66B069821928430F,"2 suspects arrested, drugs seized following No...",2.0,10F391270D3AB34C66B069821928430F-2,39E760,"Governance, Risk & Compliance Monitor Worldwide",Investigators reportedly found 26 grams of cra...,Cracking,2025-06-21
1,2025-06-21 00:00:00+00:00,EE3309C79EA552874F436912D2F6A67A,US Supreme Court sets rules for venue selectio...,3.0,EE3309C79EA552874F436912D2F6A67A-3,BC923D,Legal Monitor Worldwide,The EPA had previously argued that the CAAs na...,Oil Refinery,2025-06-21
2,2025-06-21 00:00:00+00:00,45A5868267C8A445F6B696C495AA8673,Aduro Clean Technologies Announces Closing of ...,7.0,45A5868267C8A445F6B696C495AA8673-7,D051D6,Global Data Point,About Aduro Clean Technologies\nAduro Clean Te...,Heavy Crude,2025-06-21
3,2025-06-21 00:00:00+00:00,96EC510A80FAD37155CD1543681E8CC1,"Morning Briefing: June 21, 2025",9.0,96EC510A80FAD37155CD1543681E8CC1-9,CE1ADC,Anadolu Agency,- Top European diplomats emphasized the urgenc...,Geopolitical Risk,2025-06-21
4,2025-06-21 00:00:00+00:00,86E659AE53391BE3F217C0EC3B031F97,Peter Dey Announces Retirement from Gran Tierr...,3.0,86E659AE53391BE3F217C0EC3B031F97-3,346656,Executive Appointments Worldwide,About Gran Tierra Energy Inc.\nGran Tierra Ene...,Exploration and Production,2025-06-21
...,...,...,...,...,...,...,...,...,...,...
4897,2025-06-28 23:04:15+00:00,42DCCBB6928D19107FC1113CFA419E88,"Semi-truck transporting 10,000 bottles of whis...",1.0,42DCCBB6928D19107FC1113CFA419E88-1,0C8ACA,New York Post (NYPost),They avoided a major party foul.\nA semi-truck...,Barrel,2025-06-28
4898,2025-06-28 23:14:43+00:00,96D9E1DFE1BB7A20395F38A085C20B78,Global Solar Container Power Generation System...,5.0,96D9E1DFE1BB7A20395F38A085C20B78-5,D00B10,MarketResearch.com (United States),"In terms of Japan, based on data from Fitch an...",Energy Information Administration,2025-06-28
4899,2025-06-28 23:33:44+00:00,4C596AE14BED3F44AB5D9442B44F1F5F,Better Dividend Stock: Kinder Morgan vs. Enter...,3.0,4C596AE14BED3F44AB5D9442B44F1F5F-3,648085,AOL.com,"Essentially, these midstream players sit betwe...",Midstream,2025-06-28
4900,2025-06-28 23:37:13+00:00,99083749AA42E83A4352A4002BC9479C,"3 days after Kullu flash flood, missing teen's...",2.0,99083749AA42E83A4352A4002BC9479C-2,80FC03,The Times Of India,They were swept away from a hydropower project...,Upstream,2025-06-28


In [18]:
flattened_trending_topics_df = run_process_all_trending_topics(
    unique_reports=filtered_reports,
    model=model,
    start_query=start_query,
    end_query=end_query,
    api_key=os.environ['OPENAI_API_KEY'],
    main_theme = main_theme,
    batches = 20
)


Extracting Topics for 2025-06-21:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-22:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-23:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-24:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting Topics for 2025-06-25:   0%|          | 0/20 [00:00<?, ?it/s]

...

Consolidating topic batches: 100%|██████████| 10/10 [00:23<00:00,  2.39s/it]


Summarizing text for each topic...


Generating titles: 100%|██████████| 62/62 [00:08<00:00,  7.52it/s]


Generating Day in Review summaries...
Adding one-line summaries to DataFrame...


Generating text summaries: 100%|██████████| 1713/1713 [00:42<00:00, 40.37it/s] 


**Trendiness and Novelty Scores**: We derive analytics related to the trendiness of the topic based on the news volume, and the novelty of the topic based on the changes in daily summaries, evaluating the uniqueness and freshness of each topic.

In [19]:
# Calculate trendiness and novelty scores, assessing the uniqueness and freshness of each topic
flattened_trending_topics_df = run_add_advanced_novelty_scores(flattened_trending_topics_df, api_key = os.environ['OPENAI_API_KEY'], main_theme = main_theme)

Calculating Novelty Scores:   0%|          | 0/84 [00:00<?, ?it/s]

**Financial Materiality**: We derive analytics related to the impact (Positive, Negative) and magnitude (High, Medium, Low) of the topics, inferring their  market impact on crude oil prices. The inference is based on the price mechanisms involving supply and demand dynamics, geopolitical factors among others.

In [20]:
point_of_view = "a crude oil trader, where price is influenced by supply-demand dynamics, geopolitical events, and market sentiment. \
As a trader, you focus on changes in production, inventories, and economic indicators from key markets."

flattened_trending_topics_df = add_market_impact_to_df(flattened_trending_topics_df, api_key = os.environ['OPENAI_API_KEY'], main_theme = main_theme, point_of_view = point_of_view)

We display the results of topic modeling and summarization. The **Topic** column represents the themes inferred through topic clustering using a LLM, which groups the news articles based on their content and underlying themes. The **Summary** provides a synthesized overview of all news articles within the same topic, offering a high-level view of the key messages for each cluster. The **Topic** is then rephrased into a concise form based on the summary. The **Text_Summary** provides a detailed summary of each individualchunk, capturing its core message.

In [21]:
flattened_trending_topics_df.head(5)

,Date,Day_in_Review,Topic,Summary,Source,Headline,Text,Volume_Score,Text_Summary,Volume_Score,Novelty_Score,Impact_Score,Magnitude_Score
0,2025-06-21,- **Israel-Iran Conflict**: Escalating tension...,Escalating Israel-Iran Conflict Drives 20% Sur...,The Israel-Iran conflict has led to a signific...,Business Standard via Web,India to face limited inflation impact from oi...,Brent crude prices rose 20 per cent in the las...,4,Escalating tensions between Israel and Iran ha...,4,New,Positive,High
13,2025-06-21,- **Israel-Iran Conflict**: Escalating tension...,"US-Iran Conflict Threatens Oil Supply, Could D...","US involvement in the Middle East conflict, pa...",Yahoo! Finance,Major analysts predict oil prices if Strait of...,Citi estimates that if the conflict disrupts 3...,3,Citi warns that prolonged disruptions of 3 mil...,3,Old,Positive,High
12,2025-06-21,- **Israel-Iran Conflict**: Escalating tension...,OPEC+ Production Strategies Boost Crude Oil Pr...,Recent analyses of OPEC+ production decisions ...,ZAWYA via Web,Octa's oil outlook: Middle East tensions threa...,"Overall, WTI crude price is now stuck in a bro...",2,WTI crude prices are fluctuating between $70 a...,2,New,Positive,High
11,2025-06-21,- **Israel-Iran Conflict**: Escalating tension...,OPEC+ Production Strategies Boost Crude Oil Pr...,Recent analyses of OPEC+ production decisions ...,Financial Services Monitor Worldwide,India's Core Sector Growth Drops to a Nine-Mon...,"Rahul Agrawal, Senior Economist, ICRA, said, E...",2,Despite adverse weather impacting electricity ...,2,New,Positive,High
10,2025-06-21,- **Israel-Iran Conflict**: Escalating tension...,U.S. Economic Indicators and Iran's Geopolitic...,U.S. economic indicators play a significant ro...,Business Radio KDOW,Octa's oil outlook: Middle East tensions threa...,"Iran itself, despite the existing sanctions on...",5,Iran's potential threats to close the Strait o...,5,Old,Positive,High


For verification purpose, this actionable timestamped dataframe contains the granular news clustered into relevant topics, and also the advanced analytics of trendiness, novelty, impact, and magnitude scores to be potentially used for backtesting.

# Step 4- Customized Report Generation

In this step, we rank the topics, allowing the user to customize the ranking system to reindex the news, based on their trendiness, novelty, and financial materiality on crude oil prices. We finally display a daily market update, supported by the corresponding granular news and sources for verification purpose.

The user selects the date for the report summarizing the top trending topics, and customizes the ranking system to prioritize the topics based on volume (trendiness and media attention), novelty (based on the emergence of new daily news), impact direction (positive or negative), and magnitude (financial materiality). The ranking system prioritizes the criteria in the order specified by the user, allowing for a tailored focus on the most relevant aspects of the data.

The order in which the criteria are listed in user_selected_ranking determines their priority for ranking the topics within the report. The first criterion in the list has the highest priority, followed by the second, and then the third. The user can customize the ranking by choosing to prioritize impact direction (positive or negative), novelty, magnitude, or volume, and has the flexibility to select 1, 2, or all 3 criteria based on their specific needs.

In [22]:
specific_date = '2025-06-23'  # Example date, can be modified as needed
user_selected_ranking = ['novelty', 'volume', 'magnitude']  # User can modify this list to change the ranking order
#impact_filter = 'positive_impact' #User can use the impact_filter to filter out the report

In [23]:
prepared_reports = prepare_data_for_report(flattened_trending_topics_df, user_selected_ranking, impact_filter = None, report_date = specific_date)

In [24]:
# Generate and display the HTML report for each date
for report in prepared_reports:
    html_content = generate_html_report(
        report['date'],
        report['day_in_review'],
        report['topics'],
        'Daily crude oil market update'  # Pass the main theme to dynamically generate the title
    )
    display(HTML(html_content))
    print("")
    print("")
    print("")